## Installations and Initialization

In [1]:
images_in_folder = 'images'
images_out_folder = 'images_out'
csv_out_path = 'csvs_out.csv'

In [2]:
import mediapipe as mp
import cv2
import csv
import os
import tqdm
import numpy as np

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

## CSV Header

In [4]:
num_coords = 33
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]  

## CSV_OUT

In [5]:
with open(csv_out_path, mode='w', newline='') as csv_out_file:
    csv_out_writer = csv.writer(csv_out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_out_writer.writerow(landmarks)
    # Folder names are used as pose class names.
    pose_class_names = sorted([n for n in os.listdir(images_in_folder) if not n.startswith('.')])
    
    for pose_class_name in pose_class_names:
        print('Bootstrapping ', pose_class_name)

        if not os.path.exists(os.path.join(images_out_folder, pose_class_name)):
          os.makedirs(os.path.join(images_out_folder, pose_class_name))

        image_names = sorted([
            n for n in os.listdir(os.path.join(images_in_folder, pose_class_name))
            if not n.startswith('.')])
        for image_name in tqdm.tqdm(image_names, position=0):
          # Load image.
          input_frame = cv2.imread(os.path.join(images_in_folder, pose_class_name, image_name))
          input_frame = cv2.cvtColor(input_frame, cv2.COLOR_BGR2RGB)

          # Initialize fresh pose tracker and run it.
          with mp_pose.Pose() as pose_tracker:
            result = pose_tracker.process(image=input_frame)
            pose_landmarks = result.pose_landmarks

          # Save image with pose prediction (if pose was detected).
          output_frame = input_frame.copy()
          if pose_landmarks is not None:
            mp_drawing.draw_landmarks(
                image=output_frame,
                landmark_list=pose_landmarks,
                connections=mp_pose.POSE_CONNECTIONS)
          output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
          cv2.imwrite(os.path.join(images_out_folder, image_name), output_frame)

          # Save landmarks.
          if pose_landmarks is not None:
            # Check the number of landmarks and take pose landmarks.
            assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
            pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]

            # Write pose sample to CSV.
            pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(np.str).tolist()
            csv_out_writer.writerow([pose_class_name] + pose_landmarks)


Bootstrapping  lay


  0%|                                                   | 0/200 [00:00<?, ?it/s]INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
/Users/youngseolee/Desktop/Pose_Detect/venv/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
100%|█████████████████████████████████████████| 200/200 [01:09<00:00,  2.86it/s]


Bootstrapping  sit


100%|█████████████████████████████████████████| 200/200 [01:10<00:00,  2.85it/s]


Bootstrapping  stand


100%|█████████████████████████████████████████| 200/200 [01:09<00:00,  2.87it/s]


In [6]:
cv2.imread('images/stand/image_6384.PNG')

[ WARN:0@210.730] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('images/stand/image_6384.PNG'): can't open/read file: check file path/integrity
